In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('train.tsv', sep='\t')
data_test = pd.read_csv('test.tsv', sep='\t')
data_test

,libs
0,"user32.dll,kernel32.dll"
1,"cygwin1.dll,cygiconv-2.dll,cygintl-8.dll,cygz...."
2,"winmm.dll,tapi32.dll,ws2_32.dll,setupapi.dll,v..."
3,kernel32.dll
4,kernel32.dll
...,...
1195,"kernel32.dll,msvbvm60.dll"
1196,"kernel32.dll,user32.dll,advapi32.dll"
1197,msvbvm60.dll
1198,"kernel32.dll,user32.dll,gdi32.dll,shell32.dll,..."


In [3]:
data_val = pd.read_csv('val.tsv', sep='\t')
data_val

,is_virus,filename,libs
0,1,2020-02-17T21_09_26+00_00_107385.exe,"kernel32.dll,advapi32.dll,comctl32.dll,gdi32.d..."
1,0,tac.exe,"msys-intl-8.dll,msys-2.0.dll,kernel32.dll"
2,1,VirusShare_eabeaf414d1b87659870918dc7f03f8a,kernel32.dll
3,1,VirusShare_f528efa5b5ba6643b7eb9cf4264d37e4,kernel32.dll
4,1,2019-08-14T00_52_23+00_00_60898.exe,msvbvm60.dll
...,...,...,...
1195,1,2020-02-17T21_09_26+00_00_120625.exe,mscoree.dll
1196,0,REXServerCOM.exe,"version.dll,kernel32.dll,user32.dll,advapi32.d..."
1197,1,VirusShare_0caf23c4668f7e21896d8c9393eb3f41,"kernel32.dll,gdi32.dll,msvcrt.dll,user32.dll"
1198,0,wsmprovhost.exe,"msvcrt.dll,api-ms-win-eventing-classicprovider..."


In [4]:
data_train['is_virus'][data_train['filename'].str.contains(r'VirusShare')].value_counts()

is_virus
1    6848
Name: count, dtype: int64

In [5]:
data_train['is_virus'][data_train['filename'].str.contains(r'\.exe')].value_counts()

is_virus
0    5233
1    4012
Name: count, dtype: int64

In [6]:
data_train['is_virus'][data_train['filename'].str.contains(r'\+')].value_counts()

is_virus
1    2425
0       4
Name: count, dtype: int64

In [7]:
data_train['is_virus'][data_train['filename'].str.contains(r'\-')].value_counts()

is_virus
1    4012
0     692
Name: count, dtype: int64

In [8]:
# pattern = r'\.exe'

# df_train = data_train[data_train['filename'].str.contains(pattern)].reset_index()
# df_train['filename']

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

data_train['libs'] = data_train['libs'].apply(lambda x: sorted(x.split(',')))

mlb = MultiLabelBinarizer()
enc_libs = pd.DataFrame(mlb.fit_transform(data_train['libs']), columns=mlb.classes_, index=data_train.index)

data_train = pd.concat([data_train, enc_libs], axis=1)

data_train = data_train.drop('libs', axis=1)

In [10]:
str_filenames = ['VirusShare', '.exe', '+', '-']

for j in str_filenames:
	data_train[j] = [1 if j in data_train['filename'][i] else 0 for i in range(len(data_train))]
data_train = data_train.drop(columns='filename')

In [11]:
data_val['libs'] = data_val['libs'].apply(lambda x: sorted(x.split(',')))

enc_libs = pd.DataFrame(mlb.fit_transform(data_val['libs']), columns=mlb.classes_, index=data_val.index)

data_val = pd.concat([data_val, enc_libs], axis=1)

data_val = data_val.drop('libs', axis=1)

In [12]:
for j in str_filenames:
	data_val[j] = [1 if j in data_val['filename'][i] else 0 for i in range(len(data_val))]
data_val = data_val.drop(columns='filename')

In [13]:
common_columns = data_train.columns.intersection(data_val.columns)
data_train = data_train[common_columns]
data_train

,is_virus,activeds.dll,adbwinapi.dll,advapi32.dll,api-ms-win-appmodel-runtime-l1-1-0.dll,api-ms-win-appmodel-unlock-l1-1-0.dll,api-ms-win-core-apiquery-l1-1-0.dll,api-ms-win-core-com-l1-1-0.dll,api-ms-win-core-com-l1-1-1.dll,api-ms-win-core-console-l1-1-0.dll,...,wsock32.dll,wtsapi32.dll,wttlog.dll,xinput1_3.dll,xmllite.dll,zlib1.dll,VirusShare,.exe,+,-
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16285,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
16286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16287,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
16288,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [14]:
X_train = data_train.drop(columns='is_virus')
y_train = data_train['is_virus']

In [15]:
common_columns = data_val.columns.intersection(data_train.columns)
data_val = data_val[common_columns]
data_val

,is_virus,activeds.dll,adbwinapi.dll,advapi32.dll,api-ms-win-appmodel-runtime-l1-1-0.dll,api-ms-win-appmodel-unlock-l1-1-0.dll,api-ms-win-core-apiquery-l1-1-0.dll,api-ms-win-core-com-l1-1-0.dll,api-ms-win-core-com-l1-1-1.dll,api-ms-win-core-console-l1-1-0.dll,...,wsock32.dll,wtsapi32.dll,wttlog.dll,xinput1_3.dll,xmllite.dll,zlib1.dll,VirusShare,.exe,+,-
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1196,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1197,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1198,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
X_val = data_val.drop(columns='is_virus')
y_val = data_val['is_virus']

In [17]:
from sklearn.svm import SVC

In [18]:
svm = SVC() 
svm.fit(X_train, y_train)

SVC()

In [19]:
pred = svm.predict(X_val)

In [20]:
# from sklearn.model_selection import GridSearchCV

# tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [1e-3, 1e-4], 
# 											'C': [1, 10, 100, 1000], 'class_weight': [None, 'balanced'],]

# cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3)

In [21]:
# cv.fit(X_train, y_train)

In [22]:
# cv.best_params_

In [23]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

f1_score(y_val, pred)

0.9894606323620583

In [24]:
print(classification_report(y_val, pred, digits=4))

              precision    recall  f1-score   support

           0     0.9948    0.9625    0.9784       400
           1     0.9815    0.9975    0.9895       800

    accuracy                         0.9858      1200
   macro avg     0.9882    0.9800    0.9839      1200
weighted avg     0.9860    0.9858    0.9858      1200



In [25]:
tn, fp, fn, tp = confusion_matrix(pred, y_val).ravel()
print(f'true positive: {tp}')
print(f'false positive: {fp}')
print(f'true negative: {tn}')
print(f'false negative: {fn}')

true positive: 798
false positive: 2
true negative: 385
false negative: 15


In [26]:
data_test['libs'] = data_test['libs'].apply(lambda x: sorted(x.split(',')))

enc_libs = pd.DataFrame(mlb.fit_transform(data_test['libs']), columns=mlb.classes_, index=data_test.index)

data_test = pd.concat([data_test, enc_libs], axis=1)

data_test = data_test.drop('libs', axis=1)

In [27]:
# X_train = data_train.drop(columns=['is_virus', 'VirusShare', '.exe', '+',	'-'])
y_train = data_train['is_virus']

In [28]:
common_columns = data_train.columns.intersection(data_test.columns)
data_train = data_train[common_columns]

In [29]:
common_columns = data_test.columns.intersection(data_train.columns)
data_test = data_test[common_columns]

In [30]:
svm = SVC() 
svm.fit(data_train, y_train)

SVC()

In [31]:
pred = svm.predict(data_test)

In [32]:
with open('prediction.txt', 'w') as f:
    f.write(f'{list(pred)}')